In [2]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\79165\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена: 
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ. 
  
  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size] 
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [2]:
df = pd.read_table('name_rus.txt', encoding = 'windows-1251', header = None)
df.head()

,0
0,авдокея
1,авдоким
2,авдоня
3,авдотька
4,авдотьюшка


In [3]:
class Vocab:
  def __init__(self, data):
    self.idx_to_token = dict()
    self.token_to_idx = dict()
    self.token_to_idx['<PAD>'] = 0
    self.idx_to_token[0] = '<PAD>'
    self.token_to_idx['<UNK>'] = 1
    self.idx_to_token[1] = '<UNK>'
    self.token_to_idx['<SOS>'] = 2
    self.idx_to_token[2] = '<SOS>'
    self.token_to_idx['<EOS>'] = 3
    self.idx_to_token[3] = '<EOS>'
    self.max_seq_len = 0
    k = 4
    for sur in data.values:
        for w in sur.lower():
            if w not in self.token_to_idx:
                self.token_to_idx[w] = k
                self.idx_to_token[k] = w
                k+=1
        if len(sur)>self.max_seq_len:
                self.max_seq_len = len(sur)+2
    self.vocab_len = len(self.idx_to_token)

In [4]:
vocab = Vocab(df[0])

In [5]:
class SurnamesDataset(Dataset):
  def __init__(self, vocab):
    self.vocab = vocab

  def vectorize(self, surname):
    surname_rep = []
    for sur in surname:
        rep = []
        for w in sur.lower():
            rep.append(self.vocab.token_to_idx[w])
        a = [vocab.token_to_idx['<SOS>']]+rep+[vocab.token_to_idx['<EOS>']]+[0]*(vocab.max_seq_len-len(rep)-2)
        surname_rep.append(torch.tensor(a))
    self.X_rep = torch.stack(surname_rep)
    
  def __len__(self):
    return len(self.X_rep)

  def __getitem__(self, idx):
    sample = self.X_rep[idx], torch.tensor(list(self.X_rep[idx][1:])+[0])
    return sample

In [6]:
dataset = SurnamesDataset(vocab)
dataset.vectorize(df[0].values)

In [7]:
z = next(iter(dataset))
z

(tensor([ 2,  4,  5,  6,  7,  8,  9, 10,  3,  0,  0,  0,  0,  0,  0]),
 tensor([ 4,  5,  6,  7,  8,  9, 10,  3,  0,  0,  0,  0,  0,  0,  0]))

In [8]:
data_train, data_test = torch.utils.data.random_split(dataset, [int(len(dataset)*0.81), len(dataset) - int(len(dataset)*0.81)], generator=torch.Generator().manual_seed(42))

In [9]:
train_loader = DataLoader(data_train, batch_size = 14, shuffle = True)
test_loader = DataLoader(data_test, batch_size = 14, shuffle = True)

# RNN

In [22]:
def sample_rnn(model):
    with torch.no_grad():  
        inp = [2]
        hidden = torch.zeros((1, 300))

        output_name = ''

        for i in range(vocab.max_seq_len):
            output, hidden = model(inp, hidden)
            idx = output.argmax().item()
            if idx==3:
                print(output_name)
                return
            output_name += vocab.idx_to_token[idx]
            inp = [idx]
        print(output_name)

In [23]:
losses = []

class Model(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Model, self).__init__()
    self.rnn = nn.RNN(input_size = input_size, hidden_size=hidden_size, batch_first=True)
    self.fc1 = nn.Linear(hidden_size, 100)
    self.fc2 = nn.Linear(100, output_size)
    self.emb = nn.Embedding(vocab.vocab_len, input_size)
    
  def forward(self, X, h):
    X = self.emb(torch.LongTensor(X))
    X, h = self.rnn(X, h)
    h = h.detach()
    if len(X.shape)==3:
        X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
    X = nn.ReLU()(self.fc1(F.dropout(X, 0.3)))
    y_pred = self.fc2(F.dropout(X, 0.3))
    return y_pred, h

h = torch.zeros((1, 14, 300), requires_grad=True)
model = Model(200, 300, vocab.vocab_len)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
test_losses = []

for i in range(51):
    model.train()
    losses_epoch_train = []
    losses_epoch_test = []
    y_pred_test = []
    y_true_test = []
    for x_example, y_example in train_loader:
        y_pred, h = model(x_example, h)
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_train.append(loss.item())
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
    for x_example, y_example in test_loader:
        y_pred, h = model(x_example, h)
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_test.append(loss.item())
        
        y_pred_test.extend(y_pred)
        y_true_test.extend(list(y_example))
    
    train_losses.append(np.mean(losses_epoch_train))
    test_losses.append(np.mean(losses_epoch_test))
    y_pred_test = list(map(lambda x: torch.argmax(x).item(), y_pred_test))
    y_true_test = list(map(lambda x: x.item(), torch.stack(y_true_test).reshape(-1,1)))
    if i%5==0:
        print(f'epoch {i}: train_loss = {np.mean(losses_epoch_train):.8f}, test_loss = {np.mean(losses_epoch_test):.8f}') 
        print('Точность на тестовой выборке:', accuracy_score(y_true_test, y_pred_test))
        sample_rnn(model)

epoch 0: train_loss = 1.23848795, test_loss = 1.02076060
Точность на тестовой выборке: 0.6862433862433862
марина
epoch 5: train_loss = 0.79634791, test_loss = 0.87443042
Точность на тестовой выборке: 0.7199294532627866
милина
epoch 10: train_loss = 0.71433511, test_loss = 0.82541613
Точность на тестовой выборке: 0.736331569664903
степаня
epoch 15: train_loss = 0.66994083, test_loss = 0.81562002
Точность на тестовой выборке: 0.7430335097001763
манюша
epoch 20: train_loss = 0.64580120, test_loss = 0.80504597
Точность на тестовой выборке: 0.7440917107583774
марина
epoch 25: train_loss = 0.62841837, test_loss = 0.81414111
Точность на тестовой выборке: 0.7463844797178131
митуся
epoch 30: train_loss = 0.61906879, test_loss = 0.80868988
Точность на тестовой выборке: 0.7488536155202822
марга
epoch 35: train_loss = 0.60844102, test_loss = 0.82136437
Точность на тестовой выборке: 0.7463844797178131
миленка
epoch 40: train_loss = 0.60600944, test_loss = 0.81725389
Точность на тестовой выборке: 0.

# LSTM

In [13]:
def sample_lstm():
    with torch.no_grad(): 
        inp = [2]
        h = torch.zeros((1, 100))
        c = torch.zeros((1, 100))

        output_name = ''

        for i in range(vocab.max_seq_len):
            prev_state = (h,c)
            output, (h, c) = model(inp, prev_state)
            idx = output.argmax().item()
            if idx==3:
                print(output_name)
                return
            output_name += vocab.idx_to_token[idx]
            inp = [idx]
        print(output_name)

In [16]:
losses = []

class Model(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Model, self).__init__()
    self.rnn = nn.LSTM(input_size = input_size, hidden_size=hidden_size, batch_first=True)
    self.fc1 = nn.Linear(hidden_size, 100)
    self.fc2 = nn.Linear(100, output_size)
    self.emb = nn.Embedding(vocab.vocab_len, input_size)
    
  def forward(self, X, prev_state):
    X = self.emb(torch.LongTensor(X))
    X, (h, c) = self.rnn(X, prev_state)
    h = h.detach()
    c = c.detach()
    if len(X.shape)==3:
        X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
    X = nn.ReLU()(self.fc1(F.dropout(X, 0.3)))
    y_pred = self.fc2(F.dropout(X, 0.3))
    return y_pred, (h, c)

h = torch.zeros((1, 14, 100), requires_grad=True)
c = torch.zeros((1, 14, 100), requires_grad=True)
model = Model(200, 100, vocab.vocab_len)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
test_losses = []

for i in range(51):
    model.train()
    losses_epoch_train = []
    losses_epoch_test = []
    y_pred_test = []
    y_true_test = []
    for x_example, y_example in train_loader:
        y_pred, (h, c) = model(x_example, (h, c))
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_train.append(loss.item())
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
    for x_example, y_example in test_loader:
        y_pred, (h, c) = model(x_example, (h, c))
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_test.append(loss.item())
        
        y_pred_test.extend(y_pred)
        y_true_test.extend(list(y_example))
    
    train_losses.append(np.mean(losses_epoch_train))
    test_losses.append(np.mean(losses_epoch_test))
    y_pred_test = list(map(lambda x: torch.argmax(x).item(), y_pred_test))
    y_true_test = list(map(lambda x: x.item(), torch.stack(y_true_test).reshape(-1,1)))
    if i%5==0:
        print(f'epoch {i}: train_loss = {np.mean(losses_epoch_train):.8f}, test_loss = {np.mean(losses_epoch_test):.8f}') 
        print('Точность на тестовой выборке:', accuracy_score(y_true_test, y_pred_test))
        sample_lstm()

epoch 0: train_loss = 1.49092728, test_loss = 1.10987705
Точность на тестовой выборке: 0.6724867724867725
линя
epoch 5: train_loss = 0.85844934, test_loss = 0.91496675
Точность на тестовой выборке: 0.708289241622575
ана
epoch 10: train_loss = 0.78250860, test_loss = 0.84787964
Точность на тестовой выборке: 0.7319223985890653
андрий
epoch 15: train_loss = 0.73421566, test_loss = 0.83577189
Точность на тестовой выборке: 0.7379188712522046
алексина
epoch 20: train_loss = 0.70655729, test_loss = 0.80824548
Точность на тестовой выборке: 0.7396825396825397
ма
epoch 25: train_loss = 0.68379396, test_loss = 0.81633204
Точность на тестовой выборке: 0.7393298059964727
андрина
epoch 30: train_loss = 0.66696342, test_loss = 0.79976082
Точность на тестовой выборке: 0.746031746031746
марина
epoch 35: train_loss = 0.65803130, test_loss = 0.80094813
Точность на тестовой выборке: 0.7458553791887125
милий
epoch 40: train_loss = 0.64335950, test_loss = 0.80999361
Точность на тестовой выборке: 0.747795414

# GRU

In [17]:
def sample_gru():
    with torch.no_grad():  # no need to track history in sampling
        inp = [2]
        h = torch.zeros((1, 100))

        output_name = ''

        for i in range(vocab.max_seq_len):
            output, h = model(inp, h)
            idx = output.argmax().item()
            if idx==3:
                print(output_name)
                return
            output_name += vocab.idx_to_token[idx]
            inp = [idx]
        print(output_name)

In [18]:
losses = []

class Model(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Model, self).__init__()
    self.rnn = nn.GRU(input_size = input_size, hidden_size=hidden_size, batch_first=True)
    self.fc1 = nn.Linear(hidden_size, 100)
    self.fc2 = nn.Linear(100, output_size)
    self.emb = nn.Embedding(vocab.vocab_len, input_size)
    
  def forward(self, X, h):
    X = self.emb(torch.LongTensor(X))
    X, h = self.rnn(X, h)
    h = h.detach()
    if len(X.shape)==3:
        X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
    X = nn.ReLU()(self.fc1(F.dropout(X, 0.3)))
    y_pred = self.fc2(F.dropout(X, 0.3))
    return y_pred, h

h = torch.zeros((1, 14, 100), requires_grad=True)
model = Model(200, 100, vocab.vocab_len)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
test_losses = []

for i in range(51):
    model.train()
    losses_epoch_train = []
    losses_epoch_test = []
    y_pred_test = []
    y_true_test = []
    for x_example, y_example in train_loader:
        y_pred, h = model(x_example, h)
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_train.append(loss.item())
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
    for x_example, y_example in test_loader:
        y_pred, h = model(x_example, h)
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_test.append(loss.item())
        
        y_pred_test.extend(y_pred)
        y_true_test.extend(list(y_example))
    
    train_losses.append(np.mean(losses_epoch_train))
    test_losses.append(np.mean(losses_epoch_test))
    y_pred_test = list(map(lambda x: torch.argmax(x).item(), y_pred_test))
    y_true_test = list(map(lambda x: x.item(), torch.stack(y_true_test).reshape(-1,1)))
    if i%5==0:
        print(f'epoch {i}: train_loss = {np.mean(losses_epoch_train):.8f}, test_loss = {np.mean(losses_epoch_test):.8f}') 
        print('Точность на тестовой выборке:', accuracy_score(y_true_test, y_pred_test))
        sample_gru()

epoch 0: train_loss = 1.41278142, test_loss = 1.09595696
Точность на тестовой выборке: 0.6749559082892416
лина
epoch 5: train_loss = 0.86101546, test_loss = 0.91060228
Точность на тестовой выборке: 0.7109347442680776
маня
epoch 10: train_loss = 0.77384902, test_loss = 0.85285127
Точность на тестовой выборке: 0.7268077601410935
вилина
epoch 15: train_loss = 0.73051078, test_loss = 0.83930373
Точность на тестовой выборке: 0.7358024691358025
маня
epoch 20: train_loss = 0.69961925, test_loss = 0.82560103
Точность на тестовой выборке: 0.7430335097001763
марий
epoch 25: train_loss = 0.68088812, test_loss = 0.81763531
Точность на тестовой выборке: 0.7398589065255732
милий
epoch 30: train_loss = 0.66656392, test_loss = 0.80998477
Точность на тестовой выборке: 0.7455026455026456
мильяна
epoch 35: train_loss = 0.65647179, test_loss = 0.80486397
Точность на тестовой выборке: 0.7513227513227513
васильян
epoch 40: train_loss = 0.64841724, test_loss = 0.80283762
Точность на тестовой выборке: 0.74197

## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет. 

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова) 
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3


In [3]:
f = open('The-Great-Gatsby.txt', 'r', encoding = 'utf8')
data = f.read()
print(data[:500])

CHAPTER I
In my younger and more vulnerable years my father gave me some advice that I’ve been turning over in my mind ever since.“Whenever you feel like criticizing any one,” he told me, “just remember that all the people in this world haven’t had the advantages that you’ve had.”He didn’t say any more, but we’ve always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence, I’m inclined to reserve all judgments, a habit that ha


In [102]:
class Vocab:
  def __init__(self, data):
    self.idx_to_token = dict()
    self.token_to_idx = dict()
    self.token_to_idx['<PAD>'] = 0
    self.idx_to_token[0] = '<PAD>'
    self.token_to_idx['<SOS>'] = 1
    self.idx_to_token[1] = '<SOS>'
    self.max_seq_len = 0
    k = 2
    for s in data.lower():
        if s not in self.token_to_idx:
            self.token_to_idx[s] = k
            self.idx_to_token[k] = s
            k+=1
    self.max_seq_len = 140
    self.vocab_len = len(self.idx_to_token)

In [103]:
vocab = Vocab(data)

In [104]:
class TextDataset(Dataset):
  def __init__(self, vocab):
    self.vocab = vocab

  def vectorize(self, surname):
    s_rep = []
    i = 0
    text = data.lower()
    while i < len(text)-vocab.max_seq_len:
        rep = text[i:i+vocab.max_seq_len-2]
        rep = rep[:rep.rfind(' ')+1]
        rep_new = []
        for r in rep:
            rep_new.append(vocab.token_to_idx[r])
        a = [vocab.token_to_idx['<SOS>']]+rep_new+[0]*(vocab.max_seq_len-len(rep_new)-2)
        s_rep.append(torch.tensor(a))
        i+=len(rep)
    self.X_rep = torch.stack(s_rep)

  def __len__(self):
    return len(self.X_rep)

  def __getitem__(self, idx):
    sample = self.X_rep[idx], torch.tensor(list(self.X_rep[idx][1:])+[0])
    return sample

In [105]:
dataset = TextDataset(vocab)
dataset.vectorize(data)

In [106]:
z = next(iter(dataset))
z

(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 10, 12,  9, 13, 14,  9, 14,
         15, 16, 12, 17,  7,  8,  9,  4, 12, 18,  9, 13, 15,  8,  7,  9, 19, 16,
         20, 12,  7,  8,  4, 21, 20,  7,  9, 14,  7,  4,  8, 22,  9, 13, 14,  9,
         23,  4,  6,  3,  7,  8,  9, 17,  4, 19,  7,  9, 13,  7,  9, 22, 15, 13,
          7,  9,  4, 18, 19, 10,  2,  7,  9,  6,  3,  4,  6,  9, 10, 24, 19,  7,
          9, 21,  7,  7, 12,  9,  6, 16,  8, 12, 10, 12, 17,  9, 15, 19,  7,  8,
          9, 10, 12,  9, 13, 14,  9, 13, 10, 12, 18,  9,  7, 19,  7,  8,  9,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 10, 12,  9, 13, 14,  9, 14, 15,
         16, 12, 17,  7,  8,  9,  4, 12, 18,  9, 13, 15,  8,  7,  9, 19, 16, 20,
         12,  7,  8,  4, 21, 20,  7,  9, 14,  7,  4,  8, 22,  9, 13, 14,  9, 23,
          4,  6,  3,  7,  8,  9, 17,  4, 19,  7,  9, 13,  7,  9, 22, 15, 13,  7,
          9,  4, 18, 19, 10,  2,  7,  9,  6,  

In [110]:
data_train, data_test = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)], generator=torch.Generator().manual_seed(42))

In [116]:
train_loader = DataLoader(data_train, batch_size = 4, shuffle = True)
test_loader = DataLoader(data_test, batch_size = 4, shuffle = True)

In [123]:
data_loader = DataLoader(dataset, batch_size = 20, shuffle = True)

# RNN

In [46]:
def sample_rnn(model):
    with torch.no_grad():  
        inp = [10, 9, 20, 15, 19, 7]
        hidden = torch.zeros((2, 400))

        output_name = 'i love'

        for i in range(vocab.max_seq_len):
            output, hidden = model(inp, hidden)
            idx = output[-1].argmax().item()
            if idx == 0:
                break
            output_name += vocab.idx_to_token[idx]
            inp.append(idx)
        print(output_name)

In [124]:
losses = []

class Model(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Model, self).__init__()
    self.rnn = nn.RNN(input_size = input_size, hidden_size=hidden_size, batch_first=True, num_layers=2, dropout = 0.3)
#     self.fc1 = nn.Linear(hidden_size, 100)
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.emb = nn.Embedding(vocab.vocab_len, input_size)
    
  def forward(self, X, h):
    X = self.emb(torch.LongTensor(X))
    X, h = self.rnn(X, h)
    h = h.detach()
    if len(X.shape)==3:
        X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
#     X = nn.Tanh()(self.fc1(F.dropout(X, 0.2)))
    y_pred = self.fc2(X)
    return y_pred, h

h = torch.zeros((2, 20, 400), requires_grad=True)
model = Model(100, 400, vocab.vocab_len)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
test_losses = []

for i in range(31):
    model.train()
    losses_epoch_train = []
    losses_epoch_test = []
    y_pred_test = []
    y_true_test = []
    for x_example, y_example in data_loader:
        y_pred, h = model(x_example, h)
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_train.append(loss.item())
        loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        
        optimizer.step()
        optimizer.zero_grad()
        y_pred_test.extend(y_pred)
        y_true_test.extend(list(y_example))
        
    
    train_losses.append(np.mean(losses_epoch_train))
    y_pred_test = list(map(lambda x: torch.argmax(x).item(), y_pred_test))
    y_true_test = list(map(lambda x: x.item(), torch.stack(y_true_test).reshape(-1,1)))
    if i%3==0:
        print(f'epoch {i}: train_loss = {np.mean(losses_epoch_train):.8f}') 
        print('Точность:', accuracy_score(y_true_test, y_pred_test))
        sample_rnn(model)

epoch 0: train_loss = 2.31655732
Точность: 0.34703420936720014
i lover and the was and the was and the was and the was and the was and the was and the was and the was and the was and the was and the was and th
epoch 3: train_loss = 1.64835377
Точность: 0.5038503890764939
i love to the said to the said to the said to the should and the was a little and the said to the could and the said to he was a little and the st
epoch 6: train_loss = 1.48828254
Точность: 0.5483849654969901
i loved the shook and the started to the started to the sunder the stand in the starting the porth and the starting the should have the supper of 
epoch 9: train_loss = 1.40411224
Точность: 0.5717736015269417
i loved him and the car that i was a startled to a little before the corner of the corner of the surprised the counted to the stranged and the cou
epoch 12: train_loss = 1.34362097
Точность: 0.5877294083100866
i love the stations of the world 
epoch 15: train_loss = 1.30066814
Точность: 0.5985905153428278
i l

In [125]:
for i in range(5):
    sample_rnn(model)

i love the short blowned to the sound of the shore and then the sharp of the stars. i was some one hars and the time to the sour of the shadow of 
i love there was the should got to see her head and then the short of the summer and the should over the course of the sound of the shadow of the 
i love the short of the stranger of the sharp of the sea-change in the sun and to see him so i didn’t know what you want to get up to the sunlight
i love to the sharp of the sound of the sidew and the country of the sound of the strench of the shadow and then there was a green light the sound
i love the stared to him that the count of the side, and then to the door and started to see the sound of the sun of the country of the convertain


я люблю короткий дуновение под шум берега, а затем резкий блеск звезд. я был кем-то из харса, и время для кислой тени 

мне нравится, что был должен увидеть ее голову, а затем короткий конец лета, и они должны были в течение звука тени 

я люблю короткометражку "незнакомец на берегу моря" - измениться на солнце и увидеть его так, что я не знал, чего ты хочешь, чтобы встать навстречу солнечному свету.

я люблю резкий звук стороны и страны, звук силы тени, а потом появился зеленый свет, звук

я люблю смотреть на него, что граф сбоку, а потом на дверь и начал видеть звук солнца страны обращенных.

In [127]:
torch.save(model, 'Model')

# LSTM

In [133]:
def sample_lstm(model):
    with torch.no_grad():  
        inp = [10, 9, 20, 15, 19, 7]
        h = torch.zeros((2, 300))
        c = torch.zeros((2, 300))

        output_name = 'i love'

        for i in range(vocab.max_seq_len):
            prev_state = (h,c)
            output, (h, c) = model(inp, prev_state)
            idx = output[-1].argmax().item()
            if idx == 0:
                break
            output_name += vocab.idx_to_token[idx]
            inp.append(idx)
        print(output_name)

In [134]:
losses = []

class Model(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Model, self).__init__()
    self.rnn = nn.LSTM(input_size = input_size, hidden_size=hidden_size, batch_first=True, num_layers=2, dropout = 0.2)
#     self.fc1 = nn.Linear(hidden_size, 100)
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.emb = nn.Embedding(vocab.vocab_len, input_size)
    
  def forward(self, X, prev_state):
    X = self.emb(torch.LongTensor(X))
    X, (h, c) = self.rnn(X, prev_state)
    h = h.detach()
    c = c.detach()
    if len(X.shape)==3:
        X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
#     X = nn.Tanh()(self.fc1(F.dropout(X, 0.2)))
    y_pred = self.fc2(X)
    return y_pred, (h, c)

h = torch.zeros((2, 20, 300), requires_grad=True)
c = torch.zeros((2, 20, 300), requires_grad=True)
model = Model(100, 300, vocab.vocab_len)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
test_losses = []

for i in range(31):
    model.train()
    losses_epoch_train = []
    losses_epoch_test = []
    y_pred_test = []
    y_true_test = []
    for x_example, y_example in data_loader:
        y_pred, (h, c) = model(x_example, (h, c))
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_train.append(loss.item())
        loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        
        optimizer.step()
        optimizer.zero_grad()
        
        y_pred_test.extend(y_pred)
        y_true_test.extend(list(y_example))
                
        
    
    train_losses.append(np.mean(losses_epoch_train))
    y_pred_test = list(map(lambda x: torch.argmax(x).item(), y_pred_test))
    y_true_test = list(map(lambda x: x.item(), torch.stack(y_true_test).reshape(-1,1)))
    if i%3==0:
        print(f'epoch {i}: train_loss = {np.mean(losses_epoch_train):.8f}') 
        print('Точность на тестовой выборке:', accuracy_score(y_true_test, y_pred_test))
        sample_lstm(model)

epoch 0: train_loss = 2.74131282
Точность на тестовой выборке: 0.2534906768462781
i love the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
epoch 3: train_loss = 1.80450139
Точность на тестовой выборке: 0.4642123036264866
i love the she was and the said the said the somether and the said the said the said the she said the started the said the said and the said the s
epoch 6: train_loss = 1.58672331
Точность на тестовой выборке: 0.5207862281603289
i love the short of the shoulder to the supprised the start the start of the shoulder and she was a little starting the started the staring the st
epoch 9: train_loss = 1.46710845
Точность на тестовой выборке: 0.5552672147995888
i loved the cold see of the shoulder and the back and she was a something to the counter of the course of the short of the sound of the counter an
epoch 12: train_loss = 1.38361091
Точность на тестовой выборке: 0.5779731316987227
i

In [136]:
for i in range(5):
    sample_lstm(model)

i loved him in the secret man was any other times and the sun of the steps and started to the station that he had been a sunges of the stairs and 
i loved you about the parties and the sun of the man 
i loved him.”“i don’t know what i was all this in the sunlight that they’re all the matter of a moment the past.”“i think he was a great figures o
i loved her in the summer.”“i think he was a great road concertative neightened to 
i loved herself to get the station of the man who had been there with a small 


я любила его в "тайном человеке", "в любое другое время" и "солнце ступеней", и отправилась на станцию, где он был "солнцем ступеней", и 

я любил тебя за вечеринки и солнце этого человека 

я любила его”. “я не знаю, кем я была все это время при солнечном свете, что все это дело мгновения прошлого”. “я думаю, что он был великим деятелем о

я любил ее летом”. “Я думаю, что он был отличным дорожным концертником, приближенным к 

я любила саму себя, чтобы получить место человека, который был там с маленьким

In [137]:
torch.save(model, 'Model2')

# GRU

In [171]:
def sample_gru(model):
    with torch.no_grad():  
        inp = [10, 9, 20, 15, 19, 7]
        hidden = torch.zeros((2, 300))

        output_name = 'i love'

        for i in range(vocab.max_seq_len):
            output, hidden = model(inp, hidden)
            idx = output[-1].argmax().item()
            if idx == 0:
                break
            output_name += vocab.idx_to_token[idx]
            inp.append(idx)
        print(output_name)

In [161]:
losses = []

class Model(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Model, self).__init__()
    self.rnn = nn.GRU(input_size = input_size, hidden_size=hidden_size, batch_first=True, num_layers=2, dropout = 0.3)
#     self.fc1 = nn.Linear(hidden_size, 100)
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.emb = nn.Embedding(vocab.vocab_len, input_size)
    
  def forward(self, X, h):
    X = self.emb(torch.LongTensor(X))
    X, h = self.rnn(X, h)
    h = h.detach()
    if len(X.shape)==3:
        X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
#     X = nn.Tanh()(self.fc1(F.dropout(X, 0.2)))
    y_pred = self.fc2(X)
    return y_pred, h

h = torch.zeros((2, 20, 300), requires_grad=True)
model = Model(100, 300, vocab.vocab_len)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
test_losses = []

for i in range(31):
    model.train()
    losses_epoch_train = []
    losses_epoch_test = []
    y_pred_test = []
    y_true_test = []
    for x_example, y_example in data_loader:
        y_pred, h = model(x_example, h)
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_train.append(loss.item())
        loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        
        optimizer.step()
        optimizer.zero_grad()
        y_pred_test.extend(y_pred)
        y_true_test.extend(list(y_example))
        
    
    train_losses.append(np.mean(losses_epoch_train))
    y_pred_test = list(map(lambda x: torch.argmax(x).item(), y_pred_test))
    y_true_test = list(map(lambda x: x.item(), torch.stack(y_true_test).reshape(-1,1)))
    if i%3==0:
        print(f'epoch {i}: train_loss = {np.mean(losses_epoch_train):.8f}') 
        print('Точность:', accuracy_score(y_true_test, y_pred_test))
        sample_gru(model)

epoch 0: train_loss = 2.47988492
Точность: 0.3077998825429452
i love the the was the the was the was the to to the the was the was the the the the the was and the the was the the the was and the the was to th
epoch 3: train_loss = 1.59948379
Точность: 0.5147261782410806
i loved and the contiled and the start of the was a minute and the really and the conternoon of the starded at the started the said the started th
epoch 6: train_loss = 1.42178104
Точность: 0.5635002202319777
i love and started to the started to see the started and saw the stand of the started to the started at the started at the started and the started
epoch 9: train_loss = 1.32629948
Точность: 0.5894398766700925
i loved him to the train the strange the strange the startled to the train to the train the strange of the stretting the sunderstand i don’t know 
epoch 12: train_loss = 1.25755998
Точность: 0.6090258405520481
i loved him in the sunformality of the single broke of the sunder and the concertion that he was a so

In [172]:
for i in range(5):
    sample_gru(model)

i love that he was 
i love the statement 
i love a chair 
i loved him that 
i love that the 


In [198]:
torch.save(model, 'Model3')

# LSTM v2

In [217]:
def sample_lstm(model):
    with torch.no_grad():  
        inp = [10, 9, 20, 15, 19, 7]
        h = torch.zeros((2, 300))
        c = torch.zeros((2, 300))

        output_name = 'i love'

        for i in range(vocab.max_seq_len):
            prev_state = (h,c)
            output, (h, c) = model(inp, prev_state)
            idx = output[-1].argmax().item()
            if idx == 0:
                break
            output_name += vocab.idx_to_token[idx]
            inp.append(idx)
        print(output_name)

In [218]:
losses = []

class Model(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Model, self).__init__()
    self.rnn = nn.LSTM(input_size = input_size, hidden_size=hidden_size, batch_first=True, num_layers=2, dropout = 0.2)
#     self.fc1 = nn.Linear(hidden_size, 100)
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.emb = nn.Embedding(vocab.vocab_len, input_size)
    
  def forward(self, X, prev_state):
    X = self.emb(torch.LongTensor(X))
    X, (h, c) = self.rnn(X, prev_state)
    h = h.detach()
    c = c.detach()
    if len(X.shape)==3:
        X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
#     X = nn.Tanh()(self.fc1(F.dropout(X, 0.2)))
    y_pred = self.fc2(X)
    return y_pred, (h, c)

h = torch.zeros((2, 20, 300), requires_grad=True)
c = torch.zeros((2, 20, 300), requires_grad=True)
model = Model(250, 300, vocab.vocab_len)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
test_losses = []

for i in range(31):
    model.train()
    losses_epoch_train = []
    losses_epoch_test = []
    y_pred_test = []
    y_true_test = []
    for x_example, y_example in data_loader:
        y_pred, (h, c) = model(x_example, (h, c))
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_train.append(loss.item())
        loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        
        optimizer.step()
        optimizer.zero_grad()
        
        y_pred_test.extend(y_pred)
        y_true_test.extend(list(y_example))
                
        
    
    train_losses.append(np.mean(losses_epoch_train))
    y_pred_test = list(map(lambda x: torch.argmax(x).item(), y_pred_test))
    y_true_test = list(map(lambda x: x.item(), torch.stack(y_true_test).reshape(-1,1)))
    if i%3==0:
        print(f'epoch {i}: train_loss = {np.mean(losses_epoch_train):.8f}') 
        print('Точность на тестовой выборке:', accuracy_score(y_true_test, y_pred_test))
        sample_lstm(model)

epoch 0: train_loss = 2.60666822
Точность на тестовой выборке: 0.28350095433857
i love the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
epoch 3: train_loss = 1.72257159
Точность на тестовой выборке: 0.48535824401703126
i loved the conter a monted the cars of the come and the compless of the conting the said and the said the said the came a moment of the said the 
epoch 6: train_loss = 1.53530796
Точность на тестовой выборке: 0.5348627220672442
i love the compleated the come of the started the conterney of the complete and the contering to the could start of the montered the complest of t
epoch 9: train_loss = 1.42475697
Точность на тестовой выборке: 0.5646050506533549
i loved the start of the dream of a moment of the strange of the stared the sunders of the started the strange of the startled the contruct of the
epoch 12: train_loss = 1.34903662
Точность на тестовой выборке: 0.58485171046836
i lo

In [219]:
for i in range(5):
    sample_lstm(model)

i loved him to the cournerauld realized that he was an expression that he was a great distance that he was a great floor of the country of the con
i loved him and the coupe reached the policeman book and the contracted that the sun staring the star. at his head and then arm only the cold back
i loved him and the control of the coupe and stared at the stars. the trains with the sunlight and the coupe and the contracted the controlled out
i loved her because i was something to the world and the 
i loved him and he was a great suppressions of the warm of the station of the sun of the continually and the couch and say to the 


я любил его до такой степени, что Курнеро понял, что он был выражением, что он был на большом расстоянии, что он был великим полом страны кон

я любила его, и пара дошла до книги полицейского и до того, что солнце смотрит на звезду. на его голову, а затем на руку, только холодная спина

я любила его, любила управлять купе и смотрела на звезды. поезда с солнечным светом, купе, контрактом, контролируемым выходом

я любил ее, потому что я был чем-то для мира и

я любила его, и он был великим подавителем тепла станции, солнца, кровати и, скажем,

In [220]:
for i in range(10):
    model.train()
    losses_epoch_train = []
    losses_epoch_test = []
    y_pred_test = []
    y_true_test = []
    for x_example, y_example in data_loader:
        y_pred, (h, c) = model(x_example, (h, c))
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_train.append(loss.item())
        loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        
        optimizer.step()
        optimizer.zero_grad()
        
        y_pred_test.extend(y_pred)
        y_true_test.extend(list(y_example))
                
        
    
    train_losses.append(np.mean(losses_epoch_train))
    y_pred_test = list(map(lambda x: torch.argmax(x).item(), y_pred_test))
    y_true_test = list(map(lambda x: x.item(), torch.stack(y_true_test).reshape(-1,1)))
    if i%3==0:
        print(f'epoch {i}: train_loss = {np.mean(losses_epoch_train):.8f}') 
        print('Точность на тестовой выборке:', accuracy_score(y_true_test, y_pred_test))
        sample_lstm(model)

epoch 0: train_loss = 1.06851125
Точность на тестовой выборке: 0.6627954779033916
i loved him in the steps and supper of the sound of the 
epoch 3: train_loss = 1.03295470
Точность на тестовой выборке: 0.6738694758478931
i loved the side of the world was a strange to the sound of a sort of this possibility of a short of the 
epoch 6: train_loss = 0.99957224
Точность на тестовой выборке: 0.6833284392893848
i loved his hand and the butler way of the 
epoch 9: train_loss = 0.96836158
Точность на тестовой выборке: 0.6932645720158567
i loved in a contralto invite a strain of his position of his hands so little ground. i was a straining into the sunlight and started to be a chai


In [224]:
for i in range(5):
    sample_lstm(model)

i loved into the garage with a strain to the door of the 
i loved him and then at the car.“i love here, old sport,” said tom and the chair. the only hands and the strange to the sundan cold back and the c
i loved in the door. she had dressed at the straining into the sidewalk and the cars there were all the time in a strangest of the correction of h
i loved in a sound. then i had not another warm stranger and daisy was startlingly aware of the counter of a solitary leaves in a while in the str
i loved into the door. and the reason in the room where the sun stars. at the garage with a sort of things and the lady 


я вошел в гараж с усилием, направившись к двери

я полюбил его, а потом в машине. “Мне нравится здесь, старина”, - сказал Том и кресло. единственные руки и странная для сундана холодная спина и с

я любил в дверях. она оделась, вжимаясь в тротуар, и машины там все время были в самом странном из исправлений h

я любил в звуке. тогда у меня не было другого теплого незнакомца, и Дейзи поразительно осознавала, что через некоторое время на улице появляется счетчик одиноких листьев

мне нравилось входить в дверь. и причина в комнате, где солнце светит звездами. в гараже с какими-то вещами и дамой
​

In [225]:
for i in range(10):
    model.train()
    losses_epoch_train = []
    losses_epoch_test = []
    y_pred_test = []
    y_true_test = []
    for x_example, y_example in data_loader:
        y_pred, (h, c) = model(x_example, (h, c))
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_train.append(loss.item())
        loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        
        optimizer.step()
        optimizer.zero_grad()
        
        y_pred_test.extend(y_pred)
        y_true_test.extend(list(y_example))
                
        
    
    train_losses.append(np.mean(losses_epoch_train))
    y_pred_test = list(map(lambda x: torch.argmax(x).item(), y_pred_test))
    y_true_test = list(map(lambda x: x.item(), torch.stack(y_true_test).reshape(-1,1)))
    if i%3==0:
        print(f'epoch {i}: train_loss = {np.mean(losses_epoch_train):.8f}') 
        print('Точность на тестовой выборке:', accuracy_score(y_true_test, y_pred_test))
        sample_lstm(model)

epoch 0: train_loss = 0.95676865
Точность на тестовой выборке: 0.6959844369402437
i loved him that he was a stranger and the 
epoch 3: train_loss = 0.92470929
Точность на тестовой выборке: 0.7048854793716047
i loved him out over the sunlight and started out on the stars. the word was the station in his hand was the strange to the sound of a man who had
epoch 6: train_loss = 0.89554370
Точность на тестовой выборке: 0.7137388048744677
i loved him on the sound of a commotion and the strained street and to the 
epoch 9: train_loss = 0.86840252
Точность на тестовой выборке: 0.7216231096755249
i loved daisy and in a waved 


In [237]:
for i in range(5):
    sample_lstm(model)

i loved every must have been madiessill and she looked around at the sunlight the steps and started out and took the book, and had a strange hour 
i loved every must have been college.“i know in the man was to think.”“you’re a club to tell you and stopped on the poor.”“oh, i think of course i
i loved me on the sunlight and supprosping on the stables.“i’m going to town, and she went to her hands to the past.”“hello!” i asked.“of course i
i loved by a subden’s bridge,” he said. “it was a friend of me to myself.”“come on,” she said suddenly.“the promise to her about the other car.”“s
i loved down the straightformororrow 


мне нравилось все, что должно было быть Мэдисон, и она оглянулась на солнечный свет, на ступеньки, вышла, взяла книгу и провела странный час 

я любила все, что должно было быть в колледже.“я знаю, что в мужчине было думать”. “Ты дубина, чтобы сказать тебе и остановился на бедных”. “О, я думаю, конечно, я

я любил себя на солнечном свете и прогуливался по конюшням. “Я еду в город, а она взяла себя в руки и отправилась в прошлое”. “Привет!” - спросил я.“конечно, я

я любил у моста Субдена”, - сказал он. “это был мой друг для меня самой”. “Давай”, - внезапно сказала она.”обещание, данное ей насчет другой машины“."с

я любил прямолинейную форму или печаль

In [238]:
for i in range(10):
    model.train()
    losses_epoch_train = []
    losses_epoch_test = []
    y_pred_test = []
    y_true_test = []
    for x_example, y_example in data_loader:
        y_pred, (h, c) = model(x_example, (h, c))
    
        loss = criterion(y_pred, y_example.long().reshape(1,-1)[0])
        losses_epoch_train.append(loss.item())
        loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        
        optimizer.step()
        optimizer.zero_grad()
        
        y_pred_test.extend(y_pred)
        y_true_test.extend(list(y_example))
                
        
    
    train_losses.append(np.mean(losses_epoch_train))
    y_pred_test = list(map(lambda x: torch.argmax(x).item(), y_pred_test))
    y_true_test = list(map(lambda x: x.item(), torch.stack(y_true_test).reshape(-1,1)))
    if i%3==0:
        print(f'epoch {i}: train_loss = {np.mean(losses_epoch_train):.8f}') 
        print('Точность на тестовой выборке:', accuracy_score(y_true_test, y_pred_test))
        sample_lstm(model)

epoch 0: train_loss = 0.85924008
Точность на тестовой выборке: 0.7252349141095287
i loved the pool. once of the contrasting clerk of men shaderable stretched the policeman car was 
epoch 3: train_loss = 0.82954785
Точность на тестовой выборке: 0.7332036411686977
i loved echooping tragically at the same barbeas of the 
epoch 6: train_loss = 0.80747157
Точность на тестовой выборке: 0.7398766700924975
i loved easien, and then i was still asherined as the 
epoch 9: train_loss = 0.78405950
Точность на тестовой выборке: 0.7474673322566436
i loved eyes. i had enough to do it?” he remarked at the subject of the sunlight hard. it had occurred to me that he was a gold 


In [242]:
for i in range(5):
    sample_lstm(model)

i loved each other and said gatsby, with a sort of this gray, and started up to some wild bridy.“looks here, and i was a matter of a man i knew wh
i loved eagerine at the continent, and i went over the 
i loved delicately to the station was gate.“i can’t say about it. it’s too long island,” he said with a polite toward the porch i had not been mor
i loved every money. i had a small cheerful supercilious 
i loved eyes. she was sitting on the couch and the dead sight and 


я любил друг друга и сказал “Гэтсби", с чем-то вроде этого грея, и начал с какой-то дикой брайди."Смотрите сюда, и я был человеком, которого я знал, который

мне очень понравился игерин на континенте, и я прошел через 

я любил деликатно, чтобы на станции были ворота.“я не могу сказать об этом. это слишком Лонг-айленд, - сказал он с вежливым видом в сторону крыльца, на котором я не был раньше.

я любил все деньги. у меня был маленький веселый надменный 

я любила глаза. она сидела на диване, и мертвый вид, и